In [1]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# Base Folder 및 PDF 경로 설정
base_folder = "/content/drive/MyDrive/AiffelThon/csy/Langgraph_tutorial"
pdf_folder = "/content/drive/MyDrive/AiffelThon/Input_data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install langgraph langchain langchain_openai pypdf2 sentence-transformers faiss-gpu faiss-cpu langchain-community langchain_teddynote

In [3]:
!pip install python-dotenv

In [ ]:
OPENAI_API_KEY="OPENAI_API_KEY"

In [5]:
from langgraph.graph import StateGraph

print("LangGraph successfully installed")

LangGraph successfully installed


In [6]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# .env 파일에서 환경 변수 로드
load_dotenv()

# GPT-4o-mini 설정
gpt4o_mini = ChatOpenAI(
    model_name="gpt-4o-mini",  # GPT-4o-mini에 해당하는 모델명
    temperature=0.7,
    openai_api_key=OPENAI_API_KEY,
    max_tokens=150,
)

# GPT-4o 설정
gpt4o = ChatOpenAI(
    model_name="gpt-4o",  # GPT-4o에 해당하는 모델명
    temperature=0.7,
    openai_api_key=OPENAI_API_KEY,
    max_tokens=300,
)

In [7]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from langchain.embeddings import OpenAIEmbeddings
from langchain_teddynote.graphs import visualize_graph
from langchain.schema import HumanMessage

import os
import numpy as np

In [26]:
# from PyPDF2 import PdfReader
# import os

# def extract_text_from_pdf(pdf_path):
#     reader = PdfReader(pdf_path)
#     text = ""
#     for page in reader.pages:
#         text += page.extract_text()
#     return text

# # PDF 파일 이름 리스트 생성 (paper_001.pdf ~ paper_143.pdf)
# pdf_files = [f"paper_{str(i).zfill(3)}.pdf" for i in range(1, 144)]

# # PDF 텍스트 추출 및 저장
# all_texts = []
# for pdf_file in pdf_files:
#     pdf_path = os.path.join(pdf_folder, pdf_file)
#     text = extract_text_from_pdf(pdf_path)
#     all_texts.append(text)

# print(f"총 {len(all_texts)}개의 논문 텍스트를 추출했습니다.")

In [27]:
# from langchain.text_splitter import CharacterTextSplitter

# text_splitter = CharacterTextSplitter(
#     separator="\n",
#     chunk_size=1800,
#     chunk_overlap=200
# )

# split_texts = []
# for text in all_texts:
#     split_texts.extend(text_splitter.split_text(text))

# print(f"분할된 텍스트 총 개수: {len(split_texts)}")

In [28]:
# from langchain.vectorstores import FAISS
# from langchain.embeddings import HuggingFaceEmbeddings
# import os

# # FAISS 저장 경로 설정
# faiss_folder = os.path.join(base_folder, "faiss")
# os.makedirs(faiss_folder, exist_ok=True)  # 폴더 생성

# # HuggingFace 임베딩 모델 설정
# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# # FAISS 인덱스 생성
# vectorstore = FAISS.from_texts(texts=split_texts, embedding=embedding_model)

# # FAISS 인덱스 저장 (pkl 파일 포함)
# vectorstore.save_local(folder_path=faiss_folder)

# print(f"FAISS 인덱스와 메타데이터가 {faiss_folder}에 저장되었습니다.")

In [8]:
faiss_folder = "/content/drive/MyDrive/AiffelThon/csy/Langgraph_tutorial/faiss"

In [23]:
from typing import Annotated, TypedDict

# Step 1: Define State using LangGraph
class State(TypedDict):
    messages: Annotated[list, add_messages]  # 메시지 리스트
    query: str  # Query 값을 명시적으로 포함

In [24]:
# Step 2: Initialize StateGraph
graph = StateGraph(State)

In [11]:
import huggingface_hub
huggingface_hub.login()

In [12]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.load_local(
    folder_path=faiss_folder,
    embeddings=embedding_model,  # Use the embedding associated with the LLM
    allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever()

<ipython-input-12-4b05d9577e78>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

In [13]:
# Step 5: Define RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=gpt4o_mini,
    retriever=retriever,
    return_source_documents=True
)

In [25]:
# 'answer_question' 노드 함수 정의
def answer_question(state):
    """
    Retrieve an answer using the QA chain and update the state with the response.
    """
    try:
        # 디버깅: state의 현재 상태를 출력
        print(f"Current state: {state}")

        # Query 추출
        query = state.get("query", "")
        if not query:
            raise ValueError("Query is missing in the state.")  # query가 없으면 예외 발생

        # RetrievalQA 체인 실행
        result = qa_chain({"query": query})

        # 답변 및 소스 문서 추출
        answer = result.get("result", "No answer found.")
        source_documents = result.get("source_documents", [])

        # 소스 문서 포맷팅
        if source_documents:
            sources = "\n".join(
                [doc.metadata.get("source", "Unknown Source") for doc in source_documents]
            )
        else:
            sources = "No sources available."

        # 메시지를 dict로 변환해 저장
        state["messages"].append({
            "role": "system",
            "content": f"Answer: {answer}\n\nSources:\n{sources}"
        })

        return state

    except Exception as e:
        # 에러 발생 시 메시지 업데이트
        state["messages"].append({
            "role": "system",
            "content": f"Error occurred: {str(e)}"
        })
        return state


# 노드 추가
graph.add_node("answer_question", answer_question)

# START에서 'answer_question' 노드로 엣지 추가
graph.add_edge(START, "answer_question")

# 'answer_question' 노드에서 END로 엣지 추가
graph.add_edge("answer_question", END)

# 그래프 컴파일
app = graph.compile()

# 그래프 실행 함수
def run_graph_with_query(query: str):
    """
    Run the LangGraph with a given query and print the final state messages.
    """
    if not query.strip():
        print("Query cannot be empty. Please provide a valid query.")
        return

    # 초기 상태 정의
    initial_state = {
        "messages": [],  # 메시지 리스트 초기화
        "query": query.strip(),  # 사용자가 입력한 질의
    }

    # 그래프 실행
    try:
        # 초기 상태 출력 (디버깅용)
        print("Initial state:", initial_state)

        # 그래프 실행
        final_state = app.invoke(initial_state)

        # 결과 메시지 출력
        for message in final_state["messages"]:
            if isinstance(message, dict):
                print(f"{message['role']}: {message['content']}")
            elif hasattr(message, "content"):  # SystemMessage 객체인 경우 처리
                print(f"system: {message.content}")
            else:
                print(f"Invalid message format: {message}")

    except Exception as e:
        print(f"Error during graph execution: {e}")


# 예시 실행
if __name__ == "__main__":
    # Example query
    example_query = "What is the composition of the NCM used in the paper?"
    run_graph_with_query(example_query)


Initial state: {'messages': [], 'query': 'What is the composition of the NCM used in the paper?'}
Current state: {'messages': [], 'query': 'What is the composition of the NCM used in the paper?'}


<ipython-input-25-8cee0669b392>:16: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


system: Answer: The composition of the NCM used in the paper is Ni:Co:Mn = 8:1:1, as indicated in the ICP-AES analysis.

Sources:
Unknown Source
Unknown Source
Unknown Source
Unknown Source
system: Answer: The composition of the NCM used in the paper is Ni:Co:Mn = 8:1:1, as indicated in the ICP-AES analysis.

Sources:
Unknown Source
Unknown Source
Unknown Source
Unknown Source


일단 FAISS를 통해 임베딩을 해서 그런지 Source를 모르는 것 같음.

어떻게 해야할까...?

그리고 이게 왜 LangGraph지..?
어떤 곳에서도 반복하지 않았는데... 이러면 LangChain과의 차별성이 없지 않나..?

어디에 어떻게 뭘 추가해야하지...?

너무 복잡한 코딩을 한 것 같다